+ The client first request the public key of asymmetric encryption from the server
+ Client generates a key for symmetric encryption
+ Client uses public key obtained from server to encrypt the public key and send to server
+ Server uses its own private key to decrypted the received the encrypted key from client
+ After the above negotiation process, server and client will use use symmetric encryption to encrypt the following communication with the negotiated key

In [258]:
import socket
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import padding as pad
from cryptography.hazmat.primitives import hashes
import base64
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

In [259]:
# define a special request
class Request:
    # a simple request class containing method, url and headers
    def __init__(self, method: str, url: str, headers: {str: str}, request_data: str, body):
        self.method = method  # GET, POST, PUT, DELETE
        self.url = url  # e.g. /index.html
        self.headers = headers
        self.request_data = request_data
        self.body = body

    def convert(self):
        # output request
        # custom ENCRYPTION/1.1 request
        request = "{} {} ENCRYPTION/1.1\r\n".format(self.method, self.url)
        request += "\r\n".join("{}: {}".format(k, v) for k, v in self.headers.items())
        request += "\r\n\r\n" + self.body
        return request

In [260]:
# socket connect 127.0.0.1:8080/
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(("127.0.0.1", 8080))

In [261]:
# first step: client request a public key
request = Request("GET", "/public_key", {}, "", "")
s.sendall(request.convert().encode("utf-8"))
while True:
    r = s.recv(1024)
    if r is not None:
        break
public_key = serialization.load_pem_public_key(
    r,
    backend=default_backend()
)
print("public key size: ", public_key.key_size)

public key size:  2048


In [262]:
# second step: Client generates a key for symmetric encryption
symmetric_key = os.urandom(32)
print('Key Generated', symmetric_key)

iv = os.urandom(16)
print('IV Generated', iv)

Key Generated b'b\x85\xf2\x05\x93\xb8\xe6\x98\xd2\x92\x88,iA\x04"m\xdfq\x8a\xd2\x17\x82\x1f\xeeyhq\x9b\xd8\x0bI'
IV Generated b'{oQ-\xd9\xd2\xce\xf3\xff\xfc\x1b\x80\x1d\xd4\x00\xf3'


In [263]:
# third step: Client encrypts the symmetric key with the public key and sends it to the server

# encrypt the symmetric key with the public key
encrypted_key = public_key.encrypt(
    symmetric_key,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)
encrypted_symmetric_key = base64.b64encode(encrypted_key).decode("utf-8")
print('Key Encrypted', encrypted_symmetric_key)
print('Key Encrypted Length', len(encrypted_symmetric_key))

# send the encrypted symmetric key to the server
request = Request("POST", "/encrypted_symmetric_key", {"encrypted_symmetric_key": encrypted_symmetric_key}, "", "")
s.sendall(request.convert().encode("utf-8"))

encrypted_iv = public_key.encrypt(
    iv,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)
encrypted_iv = base64.b64encode(encrypted_iv).decode("utf-8")
print('IV Encrypted', encrypted_iv)
print('IV Encrypted Length', len(encrypted_iv))
request = Request("POST", "/encrypted_iv", {"encrypted_iv": encrypted_iv}, "", "")
s.sendall(request.convert().encode("utf-8"))

Key Encrypted pOTP3zzYUskTjfJyxRB/KuS7RT48leYdYAMUCiUex7H5tzgabTjhjNdbIehQbHiboqQzySCL63SByMsfqS4LBkrh0pICdL8l1c41nibIJt6CWAso6GGeH8BnD9oPCn4KRgXZD+Zf2Uj/FUAyzJj8Zs/i6L6KZAR9Ta0l6obAht3iOKqdzvCnQdmmRiUOFjNC7wyagDJB/t7eQkRZXHxmXb82J71vLJprGY9xzTDNkYC7jTMsRgEICIgc3Rb/g6FXddX+o5rvkCUezfGg9E5bMX7B6+eiS7WuqAKUbrEQvYmHlCrOUAWfT1zyFMkCPV5E7bdskVej7+NAGZgyovs+Xw==
Key Encrypted Length 344
IV Encrypted ttOD1nSHYCmmn2tNAMIpjmB9f8HHd7fuzjnBI8aXdoZvQp9HRAyXR/Qsz7VCqcvdRXxP+gIL256LLfErBwBMEaMXDG/B0RVkW2R8GBMSwpU3CVd7SRdjGO8EKyG5UfiWqI/jQXAEGBs7hGUrBBGqx/gkn1jbJ9L7oUq+I7KU2tAKXFaq9oVQjIUbQ2ywoxu+YyTfehapSIvljUfQSe2mbVQ7W78P3K1nWnaiTJpxM/H1DPLd4F2PJf9bG19t5ABTc8LTPhPwk1oi10B/AQQ7kuMiL6XgDFfLUri86MVuGtTSjZaR9XX5iw/EnUCrF8lnTObi8CUv6k6Xk5oUW/pHhg==
IV Encrypted Length 344


In [264]:
body = """------WebKitFormBoundary7MA4YWxkTrZu0gW\r
Content-Disposition: form-data; name="firstfile"; filename="a.txt"\r
Content-Type: text/plain\r
\r
I love U!\r
------WebKitFormBoundary7MA4YWxkTrZu0gW--"""

# fourth step: Client encrypts the file with the symmetric key and sends it to the server
# encrypt the file with the symmetric key
cipher = Cipher(algorithms.AES(symmetric_key), modes.CBC(iv), backend=default_backend())
encryptor = cipher.encryptor()

# Pad the body to ensure it is a multiple of the block length
block_size = cipher.algorithm.block_size // 8
padder = pad.PKCS7(block_size * 8).padder()
padded_body = padder.update(body.encode('utf-8')) + padder.finalize()

encrypted_body = encryptor.update(padded_body) + encryptor.finalize()
encrypted_body = base64.b64encode(encrypted_body).decode("utf-8")
print('Body Encrypted', encrypted_body)

headers = {
    "Content-Type": "multipart/form-data; boundary=----WebKitFormBoundary7MA4YWxkTrZu0gW",
    "Content-Length": len(encrypted_body),
    "Authorization": "Basic Y2xpZW50MToxMjM="
}

request = Request("POST", "/upload?path=client1/", headers, "", encrypted_body)
print(request.convert())
s.sendall(request.convert().encode("utf-8"))
while True:
    r = s.recv(1024)
    if r is not None:
        break

print(r.decode("utf-8"))

Body Encrypted e/DzmdMCs7TsFiwjWkRNmbDBSW+1frop99fjoxRVDw9NRK2zN6kRrX0if0aI7JRPIZoKZnqY7v8vruxzN1C3unFPMMJI+lRvSTFkn6+6guFEvPGKJFgyqoHAsrj3L3WVZoeiXPnTt0M0LS7rKdH/OgRNQEEGabxGh9VFeEQbK9n+gtYFgyCY/vDglLz6MfH38L+AwXFz8zoTZj3M+QbERYVjajx1VSN2MBZBNxajj+Qz+BVXKOX8ekBr6YVKDvnw
POST /upload?path=client1/ ENCRYPTION/1.1
Content-Type: multipart/form-data; boundary=----WebKitFormBoundary7MA4YWxkTrZu0gW
Content-Length: 256
Authorization: Basic Y2xpZW50MToxMjM=

e/DzmdMCs7TsFiwjWkRNmbDBSW+1frop99fjoxRVDw9NRK2zN6kRrX0if0aI7JRPIZoKZnqY7v8vruxzN1C3unFPMMJI+lRvSTFkn6+6guFEvPGKJFgyqoHAsrj3L3WVZoeiXPnTt0M0LS7rKdH/OgRNQEEGabxGh9VFeEQbK9n+gtYFgyCY/vDglLz6MfH38L+AwXFz8zoTZj3M+QbERYVjajx1VSN2MBZBNxajj+Qz+BVXKOX8ekBr6YVKDvnw
HTTP/1.1 200 OK
Connection: keep-alive
Content-Type: text/html
Content-Length: 18

encrypted responseHTTP/1.1 200 OK
Connection: keep-alive
Content-Type: text/html
Content-Length: 18

encrypted response
